# 文本预处理
:label:`sec_text_preprocessing`

对于序列数据处理问题，我们在 :numref:`sec_sequence`中
评估了所需的统计工具和预测时面临的挑战。
这样的数据存在许多种形式，文本是最常见例子之一。
例如，一篇文章可以被简单地看作一串单词序列，甚至是一串字符序列。
本节中，我们将解析文本的常见预处理步骤。
这些步骤通常包括：

1. 将文本作为字符串加载到内存中。
1. 将字符串拆分为词元（如单词和字符）。
1. 建立一个词表，将拆分的词元映射到数字索引。
1. 将文本转换为数字索引序列，方便模型操作。


In [22]:
import collections
import re
from d2l import torch as d2l

## 读取数据集

首先，我们从H.G.Well的[时光机器](https://www.gutenberg.org/ebooks/35)中加载文本。
这是一个相当小的语料库，只有30000多个单词，但足够我们小试牛刀，
而现实中的文档集合可能会包含数十亿个单词。
下面的函数(**将数据集读取到由多条文本行组成的列表中**)，其中每条文本行都是一个字符串。
为简单起见，我们在这里忽略了标点符号和字母大写。


In [23]:
#@save
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():  #@save
    """将时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine'), 'r') as f:
        # 将所有文本行合并为一个列表
        lines = f.readlines()
        print(type(lines))

    # 每个文本行被转换为一个小写字符串，所有非字母字符被替换为空格
    # 并去除首尾空格
    # 最后返回一个包含所有处理后的文本行的列表
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]


# 调用函数
lines = read_time_machine()
print(type(lines))
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])

<class 'list'>
<class 'list'>
# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


## 词元化

下面的`tokenize`函数将文本行列表（`lines`）作为输入，
列表中的每个元素是一个文本序列（如一条文本行）。
[**每个文本序列又被拆分成一个词元列表**]，*词元*（token）是文本的基本单位。
最后，返回一个由词元列表组成的列表，其中的每个词元都是一个字符串（string）。


In [31]:
# 返回二维列表，第一维：句子，第二维：分词
def tokenize(lines, token='word'):  #@save
    """将文本行拆分为单词或字符词元"""
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知词元类型：' + token)

tokens = tokenize(lines,token='char')
for i in range(11):
    print(f'{i}: {tokens[i]}')

0: ['t', 'h', 'e', ' ', 't', 'i', 'm', 'e', ' ', 'm', 'a', 'c', 'h', 'i', 'n', 'e', ' ', 'b', 'y', ' ', 'h', ' ', 'g', ' ', 'w', 'e', 'l', 'l', 's']
1: []
2: []
3: []
4: []
5: ['i']
6: []
7: []
8: ['t', 'h', 'e', ' ', 't', 'i', 'm', 'e', ' ', 't', 'r', 'a', 'v', 'e', 'l', 'l', 'e', 'r', ' ', 'f', 'o', 'r', ' ', 's', 'o', ' ', 'i', 't', ' ', 'w', 'i', 'l', 'l', ' ', 'b', 'e', ' ', 'c', 'o', 'n', 'v', 'e', 'n', 'i', 'e', 'n', 't', ' ', 't', 'o', ' ', 's', 'p', 'e', 'a', 'k', ' ', 'o', 'f', ' ', 'h', 'i', 'm']
9: ['w', 'a', 's', ' ', 'e', 'x', 'p', 'o', 'u', 'n', 'd', 'i', 'n', 'g', ' ', 'a', ' ', 'r', 'e', 'c', 'o', 'n', 'd', 'i', 't', 'e', ' ', 'm', 'a', 't', 't', 'e', 'r', ' ', 't', 'o', ' ', 'u', 's', ' ', 'h', 'i', 's', ' ', 'g', 'r', 'e', 'y', ' ', 'e', 'y', 'e', 's', ' ', 's', 'h', 'o', 'n', 'e', ' ', 'a', 'n', 'd']
10: ['t', 'w', 'i', 'n', 'k', 'l', 'e', 'd', ' ', 'a', 'n', 'd', ' ', 'h', 'i', 's', ' ', 'u', 's', 'u', 'a', 'l', 'l', 'y', ' ', 'p', 'a', 'l', 'e', ' ', 'f', 'a', 'c'

## 词表

词元的类型是字符串，而模型需要的输入是数字，因此这种类型不方便模型使用。
现在，让我们[**构建一个字典，通常也叫做*词表*（vocabulary），
用来将字符串类型的词元映射到从$0$开始的数字索引中**]。
我们先将训练集中的所有文档合并在一起，对它们的唯一词元进行统计，
得到的统计结果称之为*语料*（corpus）。
然后根据每个唯一词元的出现频率，为其分配一个数字索引。
很少出现的词元通常被移除，这可以降低复杂性。
另外，语料库中不存在或已删除的任何词元都将映射到一个特定的未知词元“&lt;unk&gt;”。
我们可以选择增加一个列表，用于保存那些被保留的词元，
例如：填充词元（“&lt;pad&gt;”）；
序列开始词元（“&lt;bos&gt;”）；
序列结束词元（“&lt;eos&gt;”）。


In [32]:
class Vocab:  #@save
    """文本词表"""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 统计同名词元的频率，counter 是一个字典，键是词元，值是词元的频率
        counter = count_corpus(tokens)
        # 按出现频率排序，频率高的在前，频率低的在后
        # counter.items() 是一个有序的元组列表，每个元组是 (词元, 频率)
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True)

        # 词元代表列表，用于根据索引获取词元
        self.idx_to_token = ['<unk>'] + reserved_tokens
        # 词元代表-索引的字典，用于根据词元获取索引
        self.token_to_idx = {token: idx for idx, token in enumerate(self.idx_to_token)}

        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token) # 词元代表列表添加词元
                self.token_to_idx[token] = len(self.idx_to_token) - 1 # 词元代表-索引的字典添加词元-索引对

    def __len__(self):
        return len(self.idx_to_token)

    # 通过 vocab[token] 可以获取词元对应的索引
    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    # 通过 vocab.to_tokens(index) 可以获取索引对应的词元
    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):  # 未知词元的索引为0
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

# tokens是一个二维列表，第一维：句子，第二维：分词
# 在这个函数内部，将二维列表展平为一维列表，然后统计词元的频率
def count_corpus(tokens):  #@save
    """统计词元的频率"""
    # 如果是空列表，直接使用
    # 如果是二维列表，将其展平为一维列表
    # 如果是一维列表，直接使用
    # 这里的tokens是1D列表或2D列表
    # 二维列表（如分句后的词列表）：[['I', 'love'], ['you', 'too']]
    # 一维列表（如整个语料拍平后的词列表）：['I', 'love', 'you', 'too']
    if len(tokens) == 0 or isinstance(tokens[0], list):
        print("二维列表展平为一维列表")
        # 将词元列表展平成一个列表
        # [要放入列表的元素y  for x in 外层  for y in x]
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)
    # Counter 是 dict 的子类，用于统计可哈希对象的出现次数
    # 它返回一个字典，键是元素，值是元素的出现次数
    # 例如：Counter(['I', 'love', 'you', 'too'])
    # 输出：Counter({'I': 1, 'love': 1, 'you': 1, 'too': 1})

我们首先使用时光机器数据集作为语料库来[**构建词表**]，然后打印前几个高频词元及其索引。


In [26]:
dic = {'a': 1, 'b': 2, 'c': 3}
print(dic)
print(dic.items())
print(list(dic.items()))
print(list(dic.items())[:2])


{'a': 1, 'b': 2, 'c': 3}
dict_items([('a', 1), ('b', 2), ('c', 3)])
[('a', 1), ('b', 2), ('c', 3)]
[('a', 1), ('b', 2)]


In [34]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

二维列表展平为一维列表
[('<unk>', 0), (' ', 1), ('e', 2), ('t', 3), ('a', 4), ('i', 5), ('n', 6), ('o', 7), ('s', 8), ('h', 9)]


现在，我们可以(**将每一条文本行转换成一个数字索引列表**)。


In [28]:
for i in [0, 10]:
    print(f'i={i}')
    print('文本:', tokens[i])
    print('索引:', vocab[tokens[i]])

i=0
文本: ['t', 'h', 'e', ' ', 't', 'i', 'm', 'e', ' ', 'm', 'a', 'c', 'h', 'i', 'n', 'e', ' ', 'b', 'y', ' ', 'h', ' ', 'g', ' ', 'w', 'e', 'l', 'l', 's']
索引: [3, 9, 2, 1, 3, 5, 13, 2, 1, 13, 4, 15, 9, 5, 6, 2, 1, 21, 19, 1, 9, 1, 18, 1, 17, 2, 12, 12, 8]
i=10
文本: ['t', 'w', 'i', 'n', 'k', 'l', 'e', 'd', ' ', 'a', 'n', 'd', ' ', 'h', 'i', 's', ' ', 'u', 's', 'u', 'a', 'l', 'l', 'y', ' ', 'p', 'a', 'l', 'e', ' ', 'f', 'a', 'c', 'e', ' ', 'w', 'a', 's', ' ', 'f', 'l', 'u', 's', 'h', 'e', 'd', ' ', 'a', 'n', 'd', ' ', 'a', 'n', 'i', 'm', 'a', 't', 'e', 'd', ' ', 't', 'h', 'e']
索引: [3, 17, 5, 6, 23, 12, 2, 11, 1, 4, 6, 11, 1, 9, 5, 8, 1, 14, 8, 14, 4, 12, 12, 19, 1, 20, 4, 12, 2, 1, 16, 4, 15, 2, 1, 17, 4, 8, 1, 16, 12, 14, 8, 9, 2, 11, 1, 4, 6, 11, 1, 4, 6, 5, 13, 4, 3, 2, 11, 1, 3, 9, 2]


## 整合所有功能

在使用上述函数时，我们[**将所有功能打包到`load_corpus_time_machine`函数中**]，
该函数返回`corpus`（词元索引列表）和`vocab`（时光机器语料库的词表）。
我们在这里所做的改变是：

1. 为了简化后面章节中的训练，我们使用字符（而不是单词）实现文本词元化；
1. 时光机器数据集中的每个文本行不一定是一个句子或一个段落，还可能是一个单词，因此返回的`corpus`仅处理为单个列表，而不是使用多词元列表构成的一个列表。


In [ ]:
def load_corpus_time_machine(max_tokens=-1):  #@save
    """返回时光机器数据集的词元索引列表和词表"""
    lines = read_time_machine()      # 文本行列表
    tokens = tokenize(lines, 'char') # 大爆炸为二维列表，第一维：句子，第二维：分词
    vocab = Vocab(tokens)            # 二维词表展平为一维词表，并按照频率排序

    # 通过列表推导式将所有词元转换为数字索引，得到完整的一维语料库
    # 这个截取是基于原始文本的顺序，而不是词元的频率
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    # 返回一维语料库(数字索引)和词表
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

<class 'list'>
二维列表展平为一维列表


(170580, 28)

## 小结

* 文本是序列数据的一种最常见的形式之一。
* 为了对文本进行预处理，我们通常将文本拆分为词元，构建词表将词元字符串映射为数字索引，并将文本数据转换为词元索引以供模型操作。

## 练习

1. 词元化是一个关键的预处理步骤，它因语言而异。尝试找到另外三种常用的词元化文本的方法。
1. 在本节的实验中，将文本词元为单词和更改`Vocab`实例的`min_freq`参数。这对词表大小有何影响？


[Discussions](https://discuss.d2l.ai/t/2094)
